# The system

In [41]:
import pandas as pd
import ast

## Part 1: Loading and Understanding the Data

In [42]:
books = pd.read_csv("../data/user_data/Books.csv")
ratings = pd.read_csv("../data/user_data/Ratings.csv")
users = pd.read_csv("../data/user_data/Users.csv")
goodreads = pd.read_csv("../data/goodreads_data.csv")

/var/folders/0v/2p48l9mx23dc0422gf24z_qr0000gn/T/ipykernel_7242/3183977980.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("../data/user_data/Books.csv")


In [43]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [44]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [45]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [46]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [47]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [48]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [49]:
goodreads.head()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


In [50]:
goodreads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [51]:
goodreads[goodreads['Description'].isna()]

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
1066,1066,A Decade of Desire: Erotic Tales from the Char...,Charles Dyson,NaN,"['Contemporary', 'Epic', 'Romance', 'Erotica']",4.34,730,https://www.goodreads.com/book/show/57601103-a...
1224,1224,"Complicated Moonlight (DCYE, #2)",Lynessa Layne,NaN,"['Romance', 'Reverse Harem']",4.29,234,https://www.goodreads.com/book/show/55196363-c...
1745,1745,"Mad Love (DCYE, #3)",Lynessa Layne,NaN,"['Reverse Harem', 'Romance']",4.23,147,https://www.goodreads.com/book/show/55961601-m...
2104,2104,The Spirit of Prayer: The Believer's Authority...,Tolulope Oyewole,NaN,"['Contemporary', 'Novels']",4.45,201,https://www.goodreads.com/book/show/57392898-t...
2270,2270,Lift: Five Practices Great Managers Do Consist...,Raymond Wheeler,NaN,"['Nonfiction', 'How To', 'Business', 'Self Hel...",4.30,125,https://www.goodreads.com/book/show/55455888-lift
...,...,...,...,...,...,...,...,...
9746,9746,You'll Need a Guardian Angel,Victor Piasecki,NaN,[],3.83,6,https://www.goodreads.com/book/show/5011514-yo...
9806,9806,A Touch Of Greatness,Frank Tibolt,NaN,[],4.00,13,https://www.goodreads.com/book/show/5023105-a-...
9834,9834,Mireasma trandafirilor salbatici,Sorina Popescu,NaN,['Romanian Literature'],4.64,14,https://www.goodreads.com/book/show/18247028-m...
9850,9850,Iron Butterfly: A Novel of Africa,Clara Whaley Perkins,NaN,[],3.83,6,https://www.goodreads.com/book/show/14404443-i...


In [52]:
print("Goodreads:", goodreads.shape)
print("Books:", books.shape)
print("Users:", users.shape)
print("Ratings:", ratings.shape)

Goodreads: (10000, 8)
Books: (271360, 8)
Users: (278858, 3)
Ratings: (1149780, 3)


## Part 2: Cleaning the Data

#### Starting with the Goodreads.csv dataset

In [53]:
#The Goodreads dataset has the Genre column as a string
#Converting it to a list

goodreads['Genres'] = goodreads['Genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else[])

In [54]:
#Remove the index that came with the dataset

goodreads = goodreads.drop(columns=["Unnamed: 0"])

In [55]:
#Description has 77 missing values
#I do not want to remove those rows because I want to keep the 77 books listed in those rows
#Replacing the missing Descriptions with an empty string as it will be helpful later on

goodreads['Description'] = goodreads['Description'].fillna("")

In [59]:
#Normalize author names and book titles

goodreads['Author'] = goodreads['Author'].str.strip().str.title()
goodreads['Book'] = goodreads['Book'].str.strip().str.title()

In [60]:
goodreads.head()

#Looks good

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,To Kill A Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"[Classics, Fiction, Historical Fiction, School...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,Harry Potter And The Philosopher’S Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"[Fantasy, Fiction, Young Adult, Magic, Childre...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,Pride And Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","[Classics, Fiction, Romance, Historical Fictio...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,The Diary Of A Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"[Classics, Nonfiction, History, Biography, Mem...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"[Classics, Fiction, Dystopia, Fantasy, Politic...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


#### Now the Books.csv dataset

In [61]:
#Fix column names

books.columns = books.columns.str.replace("-","_").str.replace(" ", "_")

In [62]:
#Remove invalid publication years

before = books.shape[0]

books['Year_Of_Publication'] = pd.to_numeric(books['Year_Of_Publication'], errors='coerce')
books = books[(books['Year_Of_Publication'] >=1800) & (books['Year_Of_Publication']<= 2025)]

after = books.shape[0]

print(f"Rows removed: {before-after}")

Rows removed: 4635


In [63]:
#Normalize book author and book titles

books['Book_Author'] = books['Book_Author'].str.strip().str.title()
books['Book_Title'] = books['Book_Title'].str.strip().str.title()

In [64]:
books.head()

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_URL_S,Image_URL_M,Image_URL_L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision In Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story Of The Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies Of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [66]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266725 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 266725 non-null  object 
 1   Book_Title           266725 non-null  object 
 2   Book_Author          266723 non-null  object 
 3   Year_Of_Publication  266725 non-null  float64
 4   Publisher            266723 non-null  object 
 5   Image_URL_S          266725 non-null  object 
 6   Image_URL_M          266725 non-null  object 
 7   Image_URL_L          266725 non-null  object 
dtypes: float64(1), object(7)
memory usage: 18.3+ MB


In [67]:
books[books['Book_Author'].isna()]


,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_URL_S,Image_URL_M,Image_URL_L
118033,0751352497,A+ Quiz Masters:01 Earth,NaN,1999.0,Dorling Kindersley,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...,http://images.amazon.com/images/P/0751352497.0...
187689,9627982032,The Credit Suisse Guide To Managing Your Perso...,NaN,1995.0,Edinburgh Financial Publishing,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...,http://images.amazon.com/images/P/9627982032.0...


In [70]:
#Dropping those two rows because they do not matter

books = books.dropna(subset=["Book_Author"])

In [68]:
books[books['Publisher'].isna()]

,ISBN,Book_Title,Book_Author,Year_Of_Publication,Publisher,Image_URL_S,Image_URL_M,Image_URL_L
128890,193169656X,Tyrant Moon,Elaine Corvidae,2002.0,NaN,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...,http://images.amazon.com/images/P/193169656X.0...
129037,1931696993,Finders Keepers,Linnea Sinclair,2001.0,NaN,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...,http://images.amazon.com/images/P/1931696993.0...


In [71]:
#Also dropping those two rows because they do not matter

books = books.dropna(subset=["Publisher"])

In [73]:
books.info()

#Im happy with that, let's move on

<class 'pandas.core.frame.DataFrame'>
Index: 266721 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 266721 non-null  object 
 1   Book_Title           266721 non-null  object 
 2   Book_Author          266721 non-null  object 
 3   Year_Of_Publication  266721 non-null  float64
 4   Publisher            266721 non-null  object 
 5   Image_URL_S          266721 non-null  object 
 6   Image_URL_M          266721 non-null  object 
 7   Image_URL_L          266721 non-null  object 
dtypes: float64(1), object(7)
memory usage: 18.3+ MB


#### Cleaning the Ratings.csv dataset

In [ ]:
#Looking at the distribution of ratings

ratings['Book-Rating'].value_counts().sort_index()

Book-Rating
0     716109
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: count, dtype: int64

In [ ]:
#Dropping rows with rating = 0
#Those are instances where users clicked or viewed on the book but did not give a rating.
#Keeping the zeros will distort the algorithm 

#The dataset will still have ~400k+ ratings amount ~100k users that cover ~100k books

ratings['Book-Rating'] = pd.to_numeric(ratings['Book-Rating'], errors='coerce')
ratings = ratings[ratings['Book-Rating'] >0]

In [78]:
ratings['Book-Rating'].value_counts().sort_index()

Book-Rating
1       1770
2       2759
3       5996
4       8904
5      50974
6      36924
7      76457
8     103736
9      67541
10     78610
Name: count, dtype: int64

In [79]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 433671 entries, 1 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   User-ID      433671 non-null  int64 
 1   ISBN         433671 non-null  object
 2   Book-Rating  433671 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 13.2+ MB


In [80]:
#Fix column names

ratings.columns = ratings.columns.str.replace("-","_").str.replace(" ", "_")

In [81]:
ratings.head()

,User_ID,ISBN,Book_Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6


#### Lastly, let's clean the Users.csv dataset

In [106]:
users = pd.read_csv("../data/user_data/Users.csv")

In [107]:
#Dropping Age column as it is really messy and not relevant to the system

users = users.drop(columns=['Age'])

In [108]:
#Fix column names

users.columns = users.columns.str.replace("-","_").str.replace(" ", "_")

In [109]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   User_ID   278858 non-null  int64 
 1   Location  278858 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.3+ MB


In [110]:
#I did not anticpate using User Location as part of the system
#However, now I think it would be an interesting addition
#But only at the high level - by country

users['Country'] = users['Location'].str.split(",").str[-1].str.strip().str.lower()

In [111]:
users.head()

,User_ID,Location,Country
0,1,"nyc, new york, usa",usa
1,2,"stockton, california, usa",usa
2,3,"moscow, yukon territory, russia",russia
3,4,"porto, v.n.gaia, portugal",portugal
4,5,"farnborough, hants, united kingdom",united kingdom


In [97]:
pd.set_option('display.max_rows', None)
users['Country'].value_counts().sort_values(ascending=False)

Country
usa                                         139711
canada                                       21658
united kingdom                               18538
germany                                      17043
spain                                        13147
australia                                    11784
italy                                        11266
                                              4578
france                                        3455
portugal                                      3325
new zealand                                   3097
netherlands                                   3025
switzerland                                   1745
brazil                                        1674
china                                         1488
sweden                                        1406
india                                         1290
austria                                       1136
malaysia                                      1119
argentina              

In [113]:
#Normalize Country

users['Country'] = users['Country'].astype(str).str.lower().str.strip()

In [115]:
#Remove the stray quotes

users['Country'] = (
    users['Country']
    .str.replace('"', '', regex=False)
    .str.replace("'","", regex=False)
    .str.strip()
)

In [117]:
#Convert the blank or whitespace entries to NaN

users['Country'] = users['Country'].replace(r'^\s*$', pd.NA, regex=True)

#Convert "n/a", "na", or "none" to NaN

users['Country'] = users['Country'].replace(["n/a", "na", "none"], pd.NA)

#Filling all of the NaN as "other" so there are no missing values
users['Country']= users['Country'].fillna("other")

In [121]:
#Used ChatGPT here to find the countries that are in the native language and not English
#It included the fixes for "usa" without a specific prompt for that

country_replacements = {
    "la france": "france",
    "litalia": "italy",
    "l'italia": "italy",
    "l’italia": "italy",
    "espana": "spain",
    "deutschland": "germany",
    "suisse": "switzerland",
    "svizzera": "switzerland",
    "schweiz": "switzerland",
    "united staes": "usa",
    "united stated of america": "usa",
    "u.s.a": "usa",
    "u.s.a.": "usa",
    "españa": "spain",
    "urugua": "uruguay",
    "u.a.e": "united arab emirates",
    "l`italia": "italy",
    "england": "united kingdom",
    "scotland": "united kingdom",
    "wales": "united kingdom",
    "p r china": "china",
    "people`s republic of china": "china",
    "united kingdom.": "united kingdom",
    "burma": "myanmar",
}

users['Country'] = users['Country'].replace(country_replacements)

In [131]:
#Grouping the countries with less than 20 users as "other"
#Most of those in this list are not a country, or is gibberish

country_counts = users['Country'].value_counts()
valid_countries = country_counts[country_counts >= 20].index

users['Country_clean'] = users ['Country'].apply(
    lambda x: x if x in valid_countries else "other"
)

In [132]:
#I ran that list through ChatGPT again and it suggested fixing the following list as well
#I am adding them to "other"

territories_to_other = [
    "antarctica",
    "bermuda",
    "puerto rico",
    "hong kong",
    "macedonia",
    "yugoslavia",
]

users['Country_clean'] = users['Country_clean'].replace(
    territories_to_other, "other"
)

In [134]:
#Grouping again

country_counts = users['Country_clean'].value_counts()
valid_countries = country_counts[country_counts >= 20].index

users['Country_clean'] = users['Country_clean'].apply(
    lambda x: x if x in valid_countries else "other"
)

In [135]:
users['Country_clean'].value_counts()

Country_clean
usa                       139715
canada                     21658
united kingdom             18636
germany                    17102
spain                      13322
australia                  11785
italy                      11301
other                       6284
france                      3521
portugal                    3390
new zealand                 3120
netherlands                 3051
switzerland                 1766
brazil                      1681
china                       1492
sweden                      1456
india                       1302
austria                     1149
malaysia                    1125
argentina                   1088
finland                      943
singapore                    928
denmark                      857
mexico                       850
belgium                      823
ireland                      760
philippines                  752
turkey                       497
poland                       459
pakistan                     

In [136]:
#Looks good so let's drop the old Country column

users = users.drop(columns=['Country'])


#Lets also drop the Location column

users = users.drop(columns=('Location'))

In [137]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User_ID        278858 non-null  int64 
 1   Country_clean  278858 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.3+ MB


In [ ]:
#Finally, merging